Import Library

In [14]:
import pandas as pd
import folium
import colorsys
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

Load Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/praktikumML/tugas11'

In [4]:
df = pd.read_csv(path + '/data/dataset-wisata-jogja-sekitar.csv')
df.head()

,no,nama,type,vote_average,vote_count,htm_weekday,htm_weekend,latitude,longitude,image,description
0,9,Candi Borobudur,Budaya_Dan_Sejarah,4.7,81922,50000,50000,-7.607087,110.203623,https://res.cloudinary.com/dfciqrwpe/image/upl...,candi yang pernah masuk sebagai salah satu dar...
1,10,Candi Prambanan,Budaya_Dan_Sejarah,4.7,71751,50000,50000,-7.751835,110.491532,https://res.cloudinary.com/dfciqrwpe/image/upl...,candi prambanan adalah kompleks candi hindu te...
2,24,Tebing Breksi,Alam,4.4,51431,10000,10000,-7.781477,110.504576,https://res.cloudinary.com/dfciqrwpe/image/upl...,tebing breksi merupakan tempat wisata yang ber...
3,343,Gembira Loka Zoo,Buatan,4.5,36337,20000,25000,-7.806234,110.396798,https://res.cloudinary.com/dfciqrwpe/image/upl...,gambira loka adalah kebun binatang yang berada...
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),Budaya_Dan_Sejarah,4.6,30091,8000,8000,-7.805284,110.364203,https://res.cloudinary.com/dfciqrwpe/image/upl...,kompleks keraton merupakan museum yang menyimp...


cek data

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476 entries, 0 to 475
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   no            476 non-null    int64  
 1   nama          476 non-null    object 
 2   type          476 non-null    object 
 3   vote_average  476 non-null    float64
 4   vote_count    476 non-null    int64  
 5   htm_weekday   476 non-null    int64  
 6   htm_weekend   476 non-null    int64  
 7   latitude      476 non-null    float64
 8   longitude     476 non-null    float64
 9   image         476 non-null    object 
 10  description   476 non-null    object 
dtypes: float64(3), int64(4), object(4)
memory usage: 41.0+ KB


Kolom-kolom:

* nama: nama tempat wisata
* type: jenis wisata
* vote_average dan vote_count: rating dan jumlah vote
* htm_weekend dan htm_weekday: harga tiket saat weekend dan weekday
* latitude dan longitude: koordinat lokasi wisata

In [6]:
df.isnull().sum()

,0
no,0
nama,0
type,0
vote_average,0
vote_count,0
htm_weekday,0
htm_weekend,0
latitude,0
longitude,0
image,0


pilih fitur

In [8]:
X = df[['latitude', 'longitude']]

Standarisasi

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

menjalankan DBSCAN

In [15]:
dbscan = DBSCAN(eps=0.2, min_samples=5)
clusters = dbscan.fit_predict(X_scaled)
df['cluster'] = clusters
df.head()


,no,nama,type,vote_average,vote_count,htm_weekday,htm_weekend,latitude,longitude,image,description,cluster
0,9,Candi Borobudur,Budaya_Dan_Sejarah,4.7,81922,50000,50000,-7.607087,110.203623,https://res.cloudinary.com/dfciqrwpe/image/upl...,candi yang pernah masuk sebagai salah satu dar...,6
1,10,Candi Prambanan,Budaya_Dan_Sejarah,4.7,71751,50000,50000,-7.751835,110.491532,https://res.cloudinary.com/dfciqrwpe/image/upl...,candi prambanan adalah kompleks candi hindu te...,0
2,24,Tebing Breksi,Alam,4.4,51431,10000,10000,-7.781477,110.504576,https://res.cloudinary.com/dfciqrwpe/image/upl...,tebing breksi merupakan tempat wisata yang ber...,0
3,343,Gembira Loka Zoo,Buatan,4.5,36337,20000,25000,-7.806234,110.396798,https://res.cloudinary.com/dfciqrwpe/image/upl...,gambira loka adalah kebun binatang yang berada...,1
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),Budaya_Dan_Sejarah,4.6,30091,8000,8000,-7.805284,110.364203,https://res.cloudinary.com/dfciqrwpe/image/upl...,kompleks keraton merupakan museum yang menyimp...,1


membuat warna

In [16]:
def get_color(i):
    if i == -1:
        return "black"  # noise
    hue = (i * 0.18) % 1
    r, g, b = colorsys.hsv_to_rgb(hue, 0.75, 0.95)
    return f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}'

membuat peta jogja

In [17]:
map_wisata = folium.Map(location=[-7.8, 110.4], zoom_start=11)

In [18]:
for _, row in df.iterrows():
    warna = get_color(row['cluster'])

    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=warna,
        fill=True,
        fill_color=warna,
        fill_opacity=0.8,
        popup=f"{row['nama']} | Cluster: {row['cluster']}"
    ).add_to(map_wisata)

map_wisata